[Link to tutorial](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/stacking/)

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Documents/Personal Stuff/Repos/mike_scratch/mlj_tutorial/A-stacking`


┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.7.1). Unexpected behavior may occur.
└ @ nothing /Users/michaelherman/Documents/Personal Stuff/Repos/mike_scratch/mlj_tutorial/A-stacking/Manifest.toml:0


   Installed MLJScikitLearnInterface ─ v0.1.10
   Installed EarCut_jll ────────────── v2.2.3+0


   Installed CEnum ─────────────────── v0.4.1
   Installed TimerOutputs ──────────── v0.5.14
   Installed NetworkLayout ─────────── v0.4.4


   Installed BFloat16s ─────────────── v0.2.0
   Installed GPUArrays ─────────────── v8.1.3


...

  ✓ MLJIteration


  ✓ MLJSerialization


  ✓ MLJ


  ✓ CUDA


  ✓ EvoTrees
  61 dependencies successfully precompiled in 76 seconds. 77 already precompiled.


Stacking blends the predictions of different regressors or classifiers to gain better performance than naive averaging or majority vote.

For routine stacking tasks, use the `Stack` model ([documentation](https://alan-turing-institute.github.io/MLJ.jl/dev/composing_models/#Model-Stacking)).

Here we will build a two-model stack as an MLJ learning network, which we export as a stand-alone composite model type `MyTwoStack`. The objective here is to (i) explain how stacking works; and (ii) give an advanced demonstration of MLJ's composite model interface.

In [2]:
using MLJ
using PyPlot
using StableRNGs

In [3]:
linear = (@load LinearRegressor pkg=MLJLinearModels)()
knn = (@load KNNRegressor)()

tree_booster = (@load EvoTreeRegressor)()
forest = (@load RandomForestRegressor pkg=DecisionTree)()
svm = (@load SVMRegressor)()

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/michaelherman/.julia/packages/MLJModels/tMgLW/src/loading.jl:168


import MLJLinearModels ✔

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/michaelherman/.julia/packages/MLJModels/tMgLW/src/loading.jl:168


import NearestNeighborModels ✔


import EvoTrees ✔

...

done
Verifying transaction: ...working... 

done
Executing transaction: ...working... 

done


 ✔


SVMRegressor(
    kernel = "rbf",
    degree = 3,
    gamma = "auto",
    coef0 = 0.0,
    tol = 0.001,
    C = 1.0,
    epsilon = 0.1,
    shrinking = true,
    cache_size = 200,
    max_iter = -1)

Warm-up: define a model type to average predictions

We'll start by creating a model type `MyAverageTwo` that averages the predictions of two deterministic regressors. Here's the learning network:

In [4]:
X = source()
y = source()

model1 = linear
model2 = knn

m1 = machine(model1, X, y)
y1 = predict(m1, X)

m2 = machine(model2, X, y)
y2 = predict(m2, X)

yhat = 0.5*y1 + 0.5*y2

Node{Nothing}
  args:
    1:	Node{Nothing}
    2:	Node{Nothing}
  formula:
    +(
        #100(
            predict(
                Machine{LinearRegressor,…}, 
                Source @839)),
        #100(
            predict(
                Machine{KNNRegressor,…}, 
                Source @839)))

We wrap the learning network in a learning network machine, which specifies what source nodes are, and which node is for prediction. We use a `Deterministic` surrogate model because we're making point predictions.

In [5]:
mach = machine(Deterministic(), X, y; predict=yhat)

Machine{DeterministicSurrogate,…} trained 0 times; does not cache data
  model: DeterministicSurrogate
  args: 
    1:	Source @839 ⏎ `Nothing`
    2:	Source @350 ⏎ `Nothing`


Since we chose not to wrap our source nodes X and y in data, we can't actually fit this.

Here's the macro that "exports" the learning network as a new composite model.

In [6]:
@from_network mach begin
    mutable struct MyAverageTwo
        regressor1=model1
        regressor2=model2
    end
end

The defaults in the struct must be specified.

In [7]:
average_two = MyAverageTwo()

MyAverageTwo(
    regressor1 = LinearRegressor(
            fit_intercept = true,
            solver = nothing),
    regressor2 = KNNRegressor(
            K = 5,
            algorithm = :kdtree,
            metric = Distances.Euclidean(0.0),
            leafsize = 10,
            reorder = true,
            weights = NearestNeighborModels.Uniform()))

Evaluate the average model on the Boston data set.

In [8]:
function print_performance(model, data...)
    e = evaluate(model, data...;
                 resampling=CV(rng=StableRNG(71), nfolds=8),
                 measure=rms,
                 verbosity=0)
    μ = round(e.measurement[1], sigdigits=5)
    ste = round(std(e.per_fold[1])/sqrt(8), digits=5)
    println("$model = $μ ± $(2*ste)")
end;

X, y = @load_boston

print_performance(linear, X, y)
print_performance(knn, X, y)
print_performance(average_two, X, y)

LinearRegressor = 4.9687 ± 0.62258


KNNRegressor = 6.4682 ± 1.11394


MyAverageTwo = 5.0469 ± 0.89188


## Stacking proper

### Helper functions:

Generating folds for generating out-of-sample predictions.

In [11]:
folds(data, nfolds) = partition(1:nrows(data), (1/nfolds for i in 1:(nfolds-1))...);

In [12]:
# example
f = folds(1:10, 3)

([1, 2, 3], [4, 5, 6], [7, 8, 9, 10])

We can also use the MLJ method `restrict(X, f, i)` that restricts data X to the ith element (fold) of f, and `corestrict(X, f, i)` that restricts to the corresponding fold complement (the concatenation of all but the ith fold).

In [13]:
# example
corestrict(string.(1:10), f, 2)

7-element Vector{String}:
 "1"
 "2"
 "3"
 "7"
 "8"
 "9"
 "10"

In [14]:
restrict(string.(1:10), f, 2)

3-element Vector{String}:
 "4"
 "5"
 "6"

In [15]:
# Choose some test data and some component models (defaults for the composite model)

# <broken PyPlot code goes here>

![some plot](https://juliaai.github.io/DataScienceTutorials.jl/assets/getting-started/stacking/code/output/s1.svg)

In [16]:
# models to stack
model1 = linear
model2 = knn

KNNRegressor(
    K = 5,
    algorithm = :kdtree,
    metric = Distances.Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = NearestNeighborModels.Uniform())

In [17]:
judge = linear

LinearRegressor(
    fit_intercept = true,
    solver = nothing)

In [19]:
# Define the data nad training nodes

steps(x) = x < -3/2 ? -1 : (x < 3/2 ? 0 : 1)
x = Float64[-4, -1, 2, -3, 0, 3, -2, 1, 4]
Xraw = (x = x, )
yraw = steps.(x)
idxsort = sortperm(x)
xsort = x[idxsort]
ysort = yraw[idxsort]
step(xsort, ysort, label="truth", where="mid")


X = source(Xraw)
y = source(yraw)

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /Users/michaelherman/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Source @726 ⏎ `AbstractVector{Count}`

Our first internal node will represent the three folds (vectors of row indices) for creating out-of-sample predictions.

In [22]:
f = @node folds(X, 3)
# note: `folds` works on a table. But `X` is a _node_ representing a table. So we need to use `@node` in the syntax.

Node{Nothing}
  args:
    1:	Source @487
  formula:
    #22(
        Source @487)

In [21]:
f()

([1, 2, 3], [4, 5, 6], [7, 8, 9])

In [23]:
# We could also just overload folds to work on nodes using the `node` function.
folds(X::AbstractNode, nfolds) = node(XX->folds(XX, nfolds), X)
f = folds(X, 3)
f()

([1, 2, 3], [4, 5, 6], [7, 8, 9])

For `restrict` and `corestrict`, we'll use the overload technique.

In [31]:
MLJ.restrict(X::AbstractNode, f::AbstractNode, i) = node((XX,ff) -> restrict(XX, ff, i), X, f);
MLJ.corestrict(X::AbstractNode, f::AbstractNode, i) = node((XX,ff) -> corestrict(XX, ff, i), X, f);

In [32]:
# Now define machines for training model1 on each fold-complement.
m11 = machine(model1, MLJ.corestrict(X, f, 1), MLJ.corestrict(y, f, 1))
m12 = machine(model1, MLJ.corestrict(X, f, 2), MLJ.corestrict(y, f, 2))
m13 = machine(model1, MLJ.corestrict(X, f, 3), MLJ.corestrict(y, f, 3))

Machine{LinearRegressor,…} trained 0 times; caches data
  model: MLJLinearModels.LinearRegressor
  args: 
    1:	Node{Nothing}
    2:	Node{Nothing}


In [33]:
# Define each out-of-sample prediction of model1:
y11 = predict(m11, restrict(X, f, 1));
y12 = predict(m12, restrict(X, f, 2));
y13 = predict(m13, restrict(X, f, 3));

In [34]:
y1_oos = vcat(y11, y12, y13);

Worth noting that `vcat` and `hcat` don't need overloading to work on nodes.

Let's check our nework so far by fitting and calling y1_oos.

In [35]:
fit!(y1_oos, verbosity=0)

hcat(x,y1_oos())

9×2 Matrix{Float64}:
 -4.0  -1.44
 -1.0  -0.48
  2.0   0.48
 -3.0  -0.857143
  0.0  -2.26623e-17
  3.0   0.857143
 -2.0  -0.48
  1.0   0.48
  4.0   1.44

Now repeat the procedure for other models.

In [36]:
m21 = machine(model2, corestrict(X, f, 1), corestrict(y, f, 1))
m22 = machine(model2, corestrict(X, f, 2), corestrict(y, f, 2))
m23 = machine(model2, corestrict(X, f, 3), corestrict(y, f, 3))
y21 = predict(m21, restrict(X, f, 1));
y22 = predict(m22, restrict(X, f, 2));
y23 = predict(m23, restrict(X, f, 3));

In [37]:
y2_oos = vcat(y21, y22, y23);
fit!(y2_oos, verbosity=0)

hcat(x, y2_oos())

9×2 Matrix{Float64}:
 -4.0  -0.2
 -1.0  -0.2
  2.0   0.2
 -3.0  -0.2
  0.0   0.2
  3.0   0.2
 -2.0  -0.2
  1.0   0.2
  4.0   0.2

Now we can merge these into the adjudicator's input table and construct the machine for training the adjudicator.

In [38]:
X_oos = MLJ.table(hcat(y1_oos, y2_oos))
m_judge = machine(judge, X_oos, y)

Machine{LinearRegressor,…} trained 0 times; caches data
  model: MLJLinearModels.LinearRegressor
  args: 
    1:	Node{Nothing}
    2:	Source @726 ⏎ `AbstractVector{Count}`


When we use the stack to make predictions on new data, we will be feeding the adjudicator ordinary predictions of the base learners (rather than out-of-sample predictions). But so far, we have only defined machines to train the base learners on fold complements, not the full data, which we do now:

In [39]:
m1 = machine(model1, X, y)
m2 = machine(model2, X, y)

Machine{KNNRegressor,…} trained 0 times; caches data
  model: NearestNeighborModels.KNNRegressor
  args: 
    1:	Source @487 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @726 ⏎ `AbstractVector{Count}`
